In [1]:
import csv
from collections import defaultdict, Counter
from math import sqrt


def lire_csv(fichier):
    with open(fichier, newline="") as csvfile:
        reader = csv.DictReader(csvfile)
        data = [row for row in reader]
    return data


def diagnostiquer_colonnes(data):
    diagnostics = {}
    colonnes = data[0].keys()
    for col in colonnes:
        if col == "count":
            continue
        valeurs = [row[col] for row in data]
        try:
            valeurs_num = [float(val) for val in valeurs]
            if len(set(valeurs_num)) < 20:
                type_col = "Quantitative Discrète"
            else:
                type_col = "Quantitative Continue"
            moyenne = sum(valeurs_num) / len(valeurs_num)
            mediane = sorted(valeurs_num)[len(valeurs_num) // 2]
            ecart_type = sqrt(
                sum((x - moyenne) ** 2 for x in valeurs_num) / len(valeurs_num)
            )
            etendue = max(valeurs_num) - min(valeurs_num)
            diagnostics[col] = {
                "Type": type_col,
                "Moyenne": moyenne,
                "Médiane": mediane,
                "Écart-type": ecart_type,
                "Étendue": etendue,
            }
        except ValueError:
            if len(set(valeurs)) < 20:
                type_col = "Qualitative Nominale"
            else:
                type_col = "Qualitative Ordinale"
            mode = Counter(valeurs).most_common(1)[0][0]
            effectifs = Counter(valeurs)
            frequences = {k: v / len(valeurs) for k, v in effectifs.items()}
            diagnostics[col] = {
                "Type": type_col,
                "Mode": mode,
                "Effectifs": effectifs,
                "Fréquences": frequences,
            }
    return diagnostics


def calculer_correls(data):
    colonnes = data[0].keys()
    numeriques = {
        col: [float(row[col]) for row in data]
        for col in colonnes
        if all(row[col].replace(".", "", 1).isdigit() for row in data)
    }
    correlations = {}
    for col1 in numeriques:
        for col2 in numeriques:
            if col1 != col2:
                x = numeriques[col1]
                y = numeriques[col2]
                moyenne_x = sum(x) / len(x)
                moyenne_y = sum(y) / len(y)
                covariance = sum(
                    (xi - moyenne_x) * (yi - moyenne_y) for xi, yi in zip(x, y)
                ) / len(x)
                ecart_type_x = sqrt(sum((xi - moyenne_x) ** 2 for xi in x) / len(x))
                ecart_type_y = sqrt(sum((yi - moyenne_y) ** 2 for yi in y) / len(y))
                correlation = covariance / (ecart_type_x * ecart_type_y)
                correlations[(col1, col2)] = correlation
    return correlations


def calculer_frequences_conjointes(data, col1, col2):
    table_conjointe = defaultdict(Counter)
    for row in data:
        table_conjointe[row[col1]][row[col2]] += int(row["count"])
    total = sum(int(row["count"]) for row in data)
    frequences_conjointes = {
        k: {k2: v2 / total for k2, v2 in v.items()} for k, v in table_conjointe.items()
    }
    return table_conjointe, frequences_conjointes


def calculer_frequences_conditionnelles(data, col1, col2):
    table_conjointe = defaultdict(Counter)
    for row in data:
        table_conjointe[row[col1]][row[col2]] += int(row["count"])
    frequences_conditionnelles = {
        k: {k2: v2 / sum(v.values()) for k2, v2 in v.items()}
        for k, v in table_conjointe.items()
    }
    return frequences_conditionnelles


# Exemple d'utilisation
if __name__ == "__main__":
    # Charger les données depuis un fichier CSV
    data = lire_csv("./gb_notes_v2.csv")

    # Diagnostiquer les colonnes
    resultats = diagnostiquer_colonnes(data)

    # Afficher les résultats
    for col, details in resultats.items():
        print(f"Colonne: {col}, Type: {details['Type']}")
        if "Moyenne" in details:
            print(f"  Moyenne: {details['Moyenne']}")
            print(f"  Médiane: {details['Médiane']}")
            print(f"  Écart-type: {details['Écart-type']}")
            print(f"  Étendue: {details['Étendue']}")
        if "Mode" in details:
            print(f"  Mode: {details['Mode']}")
            print(f"  Effectifs:\n{details['Effectifs']}")
            print(f"  Fréquences:\n{details['Fréquences']}")

    # Calculer et afficher les corrélations de Pearson
    correlations = calculer_correls(data)
    print("\nCorrélations de Pearson:\n", correlations)

    # Calculer et afficher les fréquences conjointes et conditionnelles pour les colonnes qualitatives
    colonnes_qualitatives = [
        col
        for col in data[0].keys()
        if col != "count"
        and not all(row[col].replace(".", "", 1).isdigit() for row in data)
    ]
    for i in range(len(colonnes_qualitatives)):
        for j in range(i + 1, len(colonnes_qualitatives)):
            col1 = colonnes_qualitatives[i]
            col2 = colonnes_qualitatives[j]
            table_conjointe, frequences_conjointes = calculer_frequences_conjointes(
                data, col1, col2
            )
            frequences_conditionnelles = calculer_frequences_conditionnelles(
                data, col1, col2
            )

            print(
                f"\nFréquences conjointes entre {col1} et {col2}:\n",
                frequences_conjointes,
            )
            print(
                f"\nFréquences conditionnelles de {col1} par rapport à {col2}:\n",
                frequences_conditionnelles,
            )


Colonne: index, Type: Quantitative Continue
  Moyenne: 65.5
  Médiane: 66.0
  Écart-type: 38.10402428440685
  Étendue: 131.0
Colonne: math, Type: Quantitative Discrète
  Moyenne: 6.545454545454546
  Médiane: 7.0
  Écart-type: 1.9202416055516192
  Étendue: 8.0
Colonne: eng, Type: Quantitative Discrète
  Moyenne: 6.492424242424242
  Médiane: 7.0
  Écart-type: 2.057860804997665
  Étendue: 9.0
Colonne: group, Type: Qualitative Nominale
  Mode: science
  Effectifs:
Counter({'science': 74, 'lang': 58})
  Fréquences:
{'lang': 0.4393939393939394, 'science': 0.5606060606060606}
Colonne: gender, Type: Qualitative Nominale
  Mode: F
  Effectifs:
Counter({'F': 66, 'M': 66})
  Fréquences:
{'F': 0.5, 'M': 0.5}

Corrélations de Pearson:
 {('index', 'math'): 0.9827817379168441, ('index', 'eng'): 0.04748568969622526, ('index', 'count'): 0.002193524958324092, ('math', 'index'): 0.9827817379168441, ('math', 'eng'): -0.10247993446312686, ('math', 'count'): -0.004115268769983775, ('eng', 'index'): 0.047485